In [ ]:
# Cài đặt thư viện
!pip install firebase-admin
!pip install cryptography


In [3]:

import firebase_admin
from firebase_admin import credentials, db
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from firebase_admin import credentials, firestore

if not firebase_admin._apps:
    cred = credentials.Certificate("firebase_adminsdk.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://data-base-21127577-default-rtdb.asia-southeast1.firebasedatabase.app/'
    })

# Tạo khóa công khai và riêng tư
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
public_key = private_key.public_key()



In [4]:
import pandas as pd

# Đọc file CSV
df = pd.read_csv('Data.csv')
df = pd.read_csv('Data.csv', usecols=['MSSV', 'Họ Và tên', 'Điểm'])
df_cleaned = df.dropna(axis=1, how='all')  # Loại bỏ các cột nếu tất cả giá trị là NaN

# Kiểm tra lại dữ liệu sau khi đã loại bỏ các cột không cần thiết
df_cleaned.head(10)


,MSSV,Họ Và tên,Điểm
0,1653118,Phạm Quốc Bảo,1.5
1,19127200,Hoàng Duy Thành Long,8.5
2,19127396,Phan Thiên Vinh Hiển,2.8
3,19127448,Phạm Đắc Khoa,10.0
4,19127581,Huỳnh Lập Tín,9.8
5,19127620,Nguyễn Văn Vũ Tường,10.0
6,20127095,Vũ Lê Trọng Văn,9.6
7,20127119,Phạm Nguyễn Gia Bảo,0.0
8,20127322,Trần Nhật Tân,6.7
9,20127385,Huỳnh Hoàng Gia Uy,0.0


In [5]:
from firebase_admin import credentials, firestore, initialize_app

# Cấu hình Firebase
cred = credentials.Certificate('firebase_adminsdk.json')
db = firestore.client()
for index, row in df.iterrows():
    mssv_str = str(row['MSSV'])
    doc_ref = db.collection('sinh_vien').document(mssv_str)
    doc_ref.set({
        'ten': row['Họ Và tên'],
        'diem': row['Điểm']
    })


In [6]:
m_values = [7793, 7817, 7919]

def mod_inverse(a, m):
    """Tính nghịch đảo modulo."""
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

def encrypt(data, m_values):
    """Mã hóa dữ liệu sử dụng Phương pháp số dư Trung Hoa."""
    M = 1
    for m in m_values:
        M *= m

    result = 0
    for i in range(len(data)):
        Mi = M // m_values[i]
        ei = mod_inverse(Mi, m_values[i])
        result += ei * data[i] * Mi
    return result % M

def decrypt(c, m_values, index):
    """Giải mã một phần tử cụ thể của dữ liệu."""
    return c % m_values[index]

def encrypt_and_convert_to_bytes(data, m_values):
    """Mã hóa dữ liệu và chuyển đổi sang bytes."""
    encrypted_data = encrypt(data, m_values)
    return encrypted_data.to_bytes((encrypted_data.bit_length() + 7) // 8, byteorder='big')


def bytes_to_int(bytes_data):
    """Chuyển đổi bytes thành int."""
    return int.from_bytes(bytes_data, byteorder='big')




In [7]:
for index, row in df.iterrows():
    mssv_str = str(row['MSSV'])

    # Chuyển điểm số thành int sau khi nhân với 10
    diem_int = int(row['Điểm'] * 10)

    # Mã hóa điểm số và chuyển đổi sang bytes
    encrypted_score_bytes = encrypt_and_convert_to_bytes([diem_int], m_values)

    # Lưu thông tin lên Firebase
    doc_ref = db.collection('sinh_vien').document(mssv_str)
    doc_ref.set({
        'ten': row['Họ Và tên'],
        'diem_ma_hoa': encrypted_score_bytes
    })


In [9]:
def input_teacher_data():
    """Nhập thông tin từ giáo viên và gửi lên Firebase."""
    mssv = input("Nhập MSSV: ")
    ten = input("Nhập tên sinh viên: ")
    diem = float(input("Nhập điểm: "))
    diem = round(diem,1)
    diem_int = int(round(diem * 10))  # Chuyển điểm số sang int sau khi nhân với 10

    # Mã hóa điểm số và chuyển đổi sang bytes
    encrypted_score_bytes = encrypt_and_convert_to_bytes([diem_int], m_values)

    # Lưu vào Firebase
    mssv_str = str(mssv)
    doc_ref = db.collection('sinh_vien').document(mssv_str)
    doc_ref.set({
        'ten': ten,
        'diem_ma_hoa': encrypted_score_bytes
    })
    print(f"Đã mã hóa và lưu điểm cho sinh viên {ten}.")


def view_student_score():
    mssv = input("Nhập MSSV của bạn: ")
    mssv_str = str(mssv)
    doc_ref = db.collection('sinh_vien').document(mssv_str)
    doc = doc_ref.get()
    if doc.exists:
        encrypted_score = doc.to_dict()['diem_ma_hoa']
        decrypted_score = decrypt(bytes_to_int(encrypted_score), m_values, 0)

        # Chuyển đổi lại thành điểm số float và chia cho 10
        final_score = round(decrypted_score / 10, 1)
        print(f"Điểm của bạn là: {final_score}")
    else:
        print("Không tìm thấy dữ liệu cho MSSV này.")


user_role = input("Bạn là giáo viên hay sinh viên? (teacher/student): ").lower()

if user_role == 'teacher':
    input_teacher_data()
elif user_role == 'student':
    view_student_score()
else:
    print("Vai trò không xác định!")



Bạn là giáo viên hay sinh viên? (teacher/student): student
Nhập MSSV của bạn: 22147039
Điểm của bạn là: 8.7
